In [1]:
#data source https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
TRAIN_DATA = "../data/external/train.csv"
TEST_DATA = "../data/external/test.csv"

#Target column in the data. 
TARGET = "SalePrice"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#display all dataframe columns
pd.options.display.max_columns = None

In [3]:
train = pd.read_csv(TRAIN_DATA)

In [4]:
train.drop('Id', axis=1, inplace=True)